In [2]:
print("hello, world!")

hello, world!


In [3]:
# system packages 
import os
import json

# internal packages 

# external packages
from openai import OpenAI
from dotenv import load_dotenv
from graphql import build_schema, parse, build_ast_schema, validate_schema, print_ast
from jinja2 import Environment, FileSystemLoader

load_dotenv()
open_ai_api_key = os.getenv("OPENAI_API_KEY")

## GraphQL Schema Parsing

In [17]:
with open('../assets/schemas/opensea/opensea_original_schema.graphql', 'r') as schema_file:
    schema_str = schema_file.read()

parsed = parse(schema_str)
print(parsed.to_dict())

{'kind': 'document', 'definitions': [{'kind': 'enum_type_definition', 'description': None, 'name': {'kind': 'name', 'value': 'Network'}, 'directives': [], 'values': [{'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'ARBITRUM_ONE'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'ARWEAVE_MAINNET'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'AURORA'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'AVALANCHE'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'BOBA'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'BSC'}, 'directives': []}, {'kind': 'enum_value_definition', 'description': None, 'name': {'kind': 'name', 'value': 'CELO'}, 'directiv

In [18]:
# write whole schema
schema_string = print_ast(parsed)  

with open('../assets/schemas/opensea/opensea_test_schema.graphql', 'w') as schema_file:
    schema_file.write(schema_string)

In [19]:
len(parsed.definitions)

9

In [21]:
# chunk by entity
schema_string = print_ast(parsed.definitions[3]) 

# with open('../assets/schemas/opensea/opensea_test_schema.graphql', 'w') as schema_file:
#     schema_file.write(schema_string) 
schema_string

'type Marketplace @entity @regularPolling {\n  " Smart contract address of the protocol\'s main contract (Factory, Registry, etc) "\n  id: ID!\n  " Name of the NFT marketplace, for example LooksRare "\n  name: String!\n  " Slug of the NFT marketplace, for example looksrare "\n  slug: String!\n  " The blockchain network this subgraph is indexing on. "\n  network: Network!\n  " Version of the subgraph schema, in SemVer format (e.g. 1.0.0) "\n  schemaVersion: String!\n  " Version of the subgraph implementation, in SemVer format (e.g. 1.0.0) "\n  subgraphVersion: String!\n  " Version of the methodology used to compute metrics, loosely based on SemVer format (e.g. 1.0.0) "\n  methodologyVersion: String!\n  " Number of collections listed on the marketplace. "\n  collectionCount: Int!\n  " Trade count of the all collections on the marketplace. "\n  tradeCount: Int!\n  " Cumulative trade volume (in ETH) "\n  cumulativeTradeVolumeETH: BigDecimal!\n  " Revenue that goes to the marketplace protoc

In [37]:
parsed.definitions[3].directives[1].name.value # directives are the decorators for the entity (@entity)

'regularPolling'

In [39]:
parsed.definitions[3].name.value

'Marketplace'

In [28]:
print(print_ast(parsed.definitions[3]))

type Marketplace @entity @regularPolling {
  " Smart contract address of the protocol's main contract (Factory, Registry, etc) "
  id: ID!
  " Name of the NFT marketplace, for example LooksRare "
  name: String!
  " Slug of the NFT marketplace, for example looksrare "
  slug: String!
  " The blockchain network this subgraph is indexing on. "
  network: Network!
  " Version of the subgraph schema, in SemVer format (e.g. 1.0.0) "
  schemaVersion: String!
  " Version of the subgraph implementation, in SemVer format (e.g. 1.0.0) "
  subgraphVersion: String!
  " Version of the methodology used to compute metrics, loosely based on SemVer format (e.g. 1.0.0) "
  methodologyVersion: String!
  " Number of collections listed on the marketplace. "
  collectionCount: Int!
  " Trade count of the all collections on the marketplace. "
  tradeCount: Int!
  " Cumulative trade volume (in ETH) "
  cumulativeTradeVolumeETH: BigDecimal!
  " Revenue that goes to the marketplace protocol, aka protocol fee. "

In [26]:
print(print_ast(parsed.definitions[3].fields[0]))

" Smart contract address of the protocol's main contract (Factory, Registry, etc) "
id: ID!


In [62]:
# parsed.definitions[3].fields[3] == Marketplace.network: Network! - we want to find out how these are linked so that we can pull in the relevant entity
# name is the column name for the field 
print(parsed.definitions[3].fields[3].name.value)

network


In [66]:
# description is the documentation field 
print(parsed.definitions[3].fields[3].description.value)

 The blockchain network this subgraph is indexing on. 


In [68]:
print(parsed.definitions[3].fields[3].directives)

()


In [69]:
print(parsed.definitions[3].fields[3].arguments)

()


In [77]:
# so here, we find the value of the field that it is, there doesn't seem to be a clear way to say this is an entity that it references
# NonNullTypeNode at 1436:1444 | NamedTypeNode at 1436:1443
print(parsed.definitions[3].fields[3].type.type.name.value)

Network


In [84]:
print(parsed.definitions[3].fields[9].type.type.name.value)

BigDecimal


In [8]:
import json
import importlib.resources as pkg_resources

# Replace 'tokencost' with the name of your package
package_name = "tokencost"
resource_name = "model_prices.json"

try:
    # Access the JSON file within the package
    with pkg_resources.open_text(package_name, resource_name) as file:
        data = json.load(file)
    print(data)
except FileNotFoundError:
    print(f"{resource_name} not found in the package {package_name}.")


/var/folders/qc/l2x6pccx1qdbhd_r0px4crjm0000gp/T/ipykernel_54346/798662062.py:10: DeprecationWarning: open_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with pkg_resources.open_text(package_name, resource_name) as file:
Failed to update token costs. Using static costs.


{'gpt-4': {'max_tokens': 4096, 'max_input_tokens': 8192, 'max_output_tokens': 4096, 'input_cost_per_token': 3e-05, 'output_cost_per_token': 6e-05, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, 'supports_prompt_caching': True}, 'gpt-4o': {'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'output_cost_per_token': 1e-05, 'cache_read_input_token_cost': 1.25e-06, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, 'supports_parallel_function_calling': True, 'supports_response_schema': True, 'supports_vision': True, 'supports_prompt_caching': True}, 'gpt-4o-audio-preview': {'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'input_cost_per_audio_token': 0.0001, 'output_cost_per_token': 1e-05, 'output_cost_per_audio_token': 0.0002, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, '

/Users/denver/Library/Caches/pypoetry/virtualenvs/graphdoc-x8ppHhEw-py3.11/lib/python3.11/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")


In [10]:
try:
    # Access the JSON file within the package
    with pkg_resources.files(package_name).joinpath(resource_name).open('r') as file:
        data = json.load(file)
    print(data)
except FileNotFoundError:
    print(f"{resource_name} not found in the package {package_name}.")

{'gpt-4': {'max_tokens': 4096, 'max_input_tokens': 8192, 'max_output_tokens': 4096, 'input_cost_per_token': 3e-05, 'output_cost_per_token': 6e-05, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, 'supports_prompt_caching': True}, 'gpt-4o': {'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'output_cost_per_token': 1e-05, 'cache_read_input_token_cost': 1.25e-06, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, 'supports_parallel_function_calling': True, 'supports_response_schema': True, 'supports_vision': True, 'supports_prompt_caching': True}, 'gpt-4o-audio-preview': {'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'input_cost_per_audio_token': 0.0001, 'output_cost_per_token': 1e-05, 'output_cost_per_audio_token': 0.0002, 'litellm_provider': 'openai', 'mode': 'chat', 'supports_function_calling': True, '

In [11]:
data

{'gpt-4': {'max_tokens': 4096,
  'max_input_tokens': 8192,
  'max_output_tokens': 4096,
  'input_cost_per_token': 3e-05,
  'output_cost_per_token': 6e-05,
  'litellm_provider': 'openai',
  'mode': 'chat',
  'supports_function_calling': True,
  'supports_prompt_caching': True},
 'gpt-4o': {'max_tokens': 16384,
  'max_input_tokens': 128000,
  'max_output_tokens': 16384,
  'input_cost_per_token': 2.5e-06,
  'output_cost_per_token': 1e-05,
  'cache_read_input_token_cost': 1.25e-06,
  'litellm_provider': 'openai',
  'mode': 'chat',
  'supports_function_calling': True,
  'supports_parallel_function_calling': True,
  'supports_response_schema': True,
  'supports_vision': True,
  'supports_prompt_caching': True},
 'gpt-4o-audio-preview': {'max_tokens': 16384,
  'max_input_tokens': 128000,
  'max_output_tokens': 16384,
  'input_cost_per_token': 2.5e-06,
  'input_cost_per_audio_token': 0.0001,
  'output_cost_per_token': 1e-05,
  'output_cost_per_audio_token': 0.0002,
  'litellm_provider': 'opena

## OpenAI Directly

In [3]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

def gpt_chat_completion(client, message, model="gpt-4o"):
    return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": message,
        }
    ],
    model=model,
)

rep = gpt_chat_completion(client, "Hello! How are you?")
print(rep.choices[0].message.content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?


## Initial Test Set 

### Prompts 

- [ ] One entity quality vs. another entity quality 
- [ ] One column quality vs. another column quality 
- [ ] One schema quality vs. another schema quality 

### Demonstrations 

- [ ] A high quality entity (description)
- [ ] A high quality column (description)
- [ ] A high quality schema (description)

### Quality Categories

- [ ] 

In [4]:
with open("tests/assets/entity_comparison_assets.json", "r") as f:
    entity_comparison_assets = json.load(f)

gold_entity_comparison = "".join(entity_comparison_assets["gold"]["prompt"])
four_entity_comparison = "".join(entity_comparison_assets["four"]["prompt"])
three_entity_comparison = "".join(entity_comparison_assets["three"]["prompt"])
two_entity_comparison = "".join(entity_comparison_assets["two"]["prompt"])
one_entity_comparison = "".join(entity_comparison_assets["one"]["prompt"])

In [5]:
# Function to reload the template
def reload_template(template_path="../assets/prompts", template_name="entity_comparison_prompt.txt"):
    env = Environment(loader=FileSystemLoader(template_path))
    env.cache.clear()
    entity_comparison_prompt_template = env.get_template(template_name)
    return entity_comparison_prompt_template

def parse_response(response): 
    response = response.choices[0].message.content
    response = response.strip('```json').strip('```').strip()
    response = json.loads(response)
    return response

example_prompt = reload_template()
example_prompt_filled = example_prompt.render({"entity_pred": "<example prediction goes here>", "entity_gold": "<example gold goes here>"})

entity_comparison_prompt_template = reload_template()
gold_output = entity_comparison_prompt_template.render({"entity_pred": {gold_entity_comparison}, "entity_gold": {gold_entity_comparison}})
four_output = entity_comparison_prompt_template.render({"entity_pred": {four_entity_comparison}, "entity_gold": {gold_entity_comparison}})
three_output = entity_comparison_prompt_template.render({"entity_pred": {three_entity_comparison}, "entity_gold": {gold_entity_comparison}})
two_output = entity_comparison_prompt_template.render({"entity_pred": {two_entity_comparison}, "entity_gold": {gold_entity_comparison}})
one_output = entity_comparison_prompt_template.render({"entity_pred": {one_entity_comparison}, "entity_gold": {gold_entity_comparison}})

print("Expecting output: 3 \n")
gold_comparison = parse_response(gpt_chat_completion(client, gold_output))
print(gold_comparison)

print("\n Expecting output: 4 \n")
four_comparison = parse_response(gpt_chat_completion(client, four_output))
print(four_comparison)

print("\n Expecting output: 3 \n")
three_comparison = parse_response(gpt_chat_completion(client, three_output))
print(three_comparison)

print("\n Expecting output: 2 \n")
two_comparison = parse_response(gpt_chat_completion(client, two_output))
print(two_comparison)

print("\n Expecting output: 1 \n")
one_comparison = parse_response(gpt_chat_completion(client, one_output))
print(one_comparison)  

Expecting output: 3 

{'reasoning': "To evaluate the documentation provided, I will compare it against the GOLD documentation and the criteria provided. The task requires determining whether the evaluated documentation matches or exceeds the quality of the GOLD documentation, based on criteria of correctness, clarity, and descriptiveness.\n\nFirst, we need to examine the documentation submitted for evaluation:\n\n- The 'id' field contains a clear and concise description, matching the GOLD example.\n- Similarly, the 'marketplace' field is well-described, stating which marketplace the snapshot belongs to, consistent with the GOLD documentation.\n- The 'blockNumber' and 'timestamp' fields are described accurately, specifying where and when the snapshot is taken, aligning with the GOLD standard.\n- The 'collectionCount', 'cumulativeTradeVolumeETH', 'marketplaceRevenueETH', 'creatorRevenueETH', 'totalRevenueETH', 'tradeCount', 'cumulativeUniqueTraders', 'dailyActiveTraders', 'dailyTradedCol

In [6]:
entity_comparison_prompt_template = reload_template(
    template_path="../assets/prompts",
    template_name="entity_comparison_revision.txt",
)

entity_comparison_prompt_modification = entity_comparison_prompt_template.render(
    {
        "original_prompt": {example_prompt_filled}, 

        "four_result_correct": {"True" if four_comparison["correctness"] == 4 else "False"},
        "four_result": {four_comparison["reasoning"]},
        
        "three_result_correct": {"True" if three_comparison["correctness"] == 3 else "False"},
        "three_result": {three_comparison["reasoning"]},
        
        "two_result_correct": {"True" if two_comparison["correctness"] == 2 else "False"},
        "two_result": {two_comparison["reasoning"]},
        
        "one_result_correct": {"True" if one_comparison["correctness"] == 1 else "False"},
        "one_result": {one_comparison["reasoning"]},
    }
)

prompt_revision = gpt_chat_completion(client, entity_comparison_prompt_modification)

In [7]:
file_path = "../assets/prompts/entity_comparison_prompt_revised.txt"

prompt_revision = parse_response(prompt_revision)
revised_prompt = prompt_revision["modified_prompt"]

# Replace the placeholder with Jinja syntax
revised_prompt = revised_prompt.replace(r"{entity_pred}", r"{{ entity_pred }}")
revised_prompt = revised_prompt.replace(r"{entity_gold}", r"{{ entity_gold }}")

with open(file_path, "w") as file:
    file.write(revised_prompt)

## Json formatting

In [12]:
def format_schema_to_json(schema_text):
    """
    Convert GraphQL-like schema text into a specific JSON format.
    
    Args:
        schema_text (str): The input schema text
        
    Returns:
        dict: Formatted JSON object
    """
    # Split the input text into lines and clean them
    lines = schema_text.strip().split('\n')
    
    # Initialize the formatted lines list
    formatted_lines = []
    
    for line in lines:
        # Clean the line and add proper escaping
        cleaned_line = line.replace('"', '\"').rstrip()
        
        # Skip empty lines that don't add value
        if not cleaned_line:
            formatted_lines.append("\n")
            continue
            
        # Add the line with proper formatting
        formatted_lines.append(cleaned_line + " \n")
    
    # Create the JSON structure
    json_output = {
        "gold": {
            "version": "1.0.0",
            "prompt": formatted_lines
        }
    }
    
    return json_output

# Example usage function to demonstrate the formatter
def example_usage(schema_text):
    """
    Demonstrate the usage of the schema formatter with pretty printing
    """
    import json
    
    # Format the schema
    formatted_json = format_schema_to_json(schema_text)
    
    # Pretty print the result
    print(json.dumps(formatted_json, indent=4))

# Test with a sample input
sample_schema = '''
type Collection @entity @regularPolling {
  " Contract address. "
  id: ID!

  " Collection name, mirrored from the smart contract. Leave null if not available. "
  name: String
}
'''

# if __name__ == "__main__":
example_usage(sample_schema)

{
    "gold": {
        "version": "1.0.0",
        "prompt": [
            "type Collection @entity @regularPolling { \n",
            "  \" Contract address. \" \n",
            "  id: ID! \n",
            "\n",
            "  \" Collection name, mirrored from the smart contract. Leave null if not available. \" \n",
            "  name: String \n",
            "} \n"
        ]
    }
}


In [6]:
from graphdoc import Parser 
from graphql import build_schema, parse, build_ast_schema, validate_schema, print_ast

In [7]:
par = Parser(
    schema_directory_path="../assets/schemas/opensea",
)

In [9]:
ast_text_example = print_ast(par.parse_schema_from_file("opensea_original_schema.graphql"))

In [10]:
par.format_text_schema_to_json(ast_text_example)    

{'gold': {'version': '1.0.0',
  'prompt': ['\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '} \n']}}

In [15]:
import json
with open("test_schema_output.json", "w") as f:
    json_output_ex = format_schema_to_json(ast_text_example)
    json.dump(json_output_ex, f, indent=4)

In [16]:
print(json_output_ex)   

{'gold': {'version': '1.0.0', 'prompt': ['enum Network { \n', '  ARBITRUM_ONE \n', '  ARWEAVE_MAINNET \n', '  AURORA \n', '  AVALANCHE \n', '  BOBA \n', '  BSC \n', '  CELO \n', '  COSMOS \n', '  CRONOS \n', '  MAINNET \n', '  FANTOM \n', '  FUSE \n', '  HARMONY \n', '  JUNO \n', '  MOONBEAM \n', '  MOONRIVER \n', '  NEAR_MAINNET \n', '  OPTIMISM \n', '  OSMOSIS \n', '  MATIC \n', '  XDAI \n', '} \n', '\n', 'enum NftStandard { \n', '  ERC721 \n', '  ERC1155 \n', '  UNKNOWN \n', '} \n', '\n', 'enum SaleStrategy { \n', '  " Strategy that executes an order at a fixed price that can be taken either by a bid or an ask. " \n', '  STANDARD_SALE \n', '  " Strategy that executes an order at a fixed price that can be matched by any tokenId for the collection. " \n', '  ANY_ITEM_FROM_COLLECTION \n', '  " Strategy that executes an order at a fixed price that can be matched by any tokenId in a set of tokenIds. " \n', '  ANY_ITEM_FROM_SET \n', '  " Strategy to launch a Dutch Auction for a token wher

## Subgraph Metadata Collection

In [ ]:
# https://api.thegraph.com/ipfs/api/v0/cat?arg=QmcQY5m381fPQUS6cEFY4VFZadtarYXNfRbEWJ7G7gbq7T
# https://gateway.thegraph.com/api/[api-key]/subgraphs/id/DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp

"""graphql
{
  subgraph(id:"DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp") {
    id
    metadata {
      codeRepository
    }
    currentVersion {
      subgraphDeployment {
        ipfsHash
      }
    }
  }
}
"""

In [53]:
import os
from subgrounds.pagination import ShallowStrategy
from subgrounds import Subgrounds
from dotenv import load_dotenv
load_dotenv()

GRAPH_API_KEY=os.getenv("GRAPH_API_KEY")
subgraph_url = f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp"

sg = Subgrounds()
network_subgraph = sg.load_subgraph(subgraph_url)

/var/folders/qc/l2x6pccx1qdbhd_r0px4crjm0000gp/T/ipykernel_59538/3513027921.py:10: ResourceWarning: unclosed <ssl.SSLSocket fd=81, family=2, type=1, proto=0, laddr=('192.168.1.205', 62095), raddr=('104.18.40.31', 443)>
  sg = Subgrounds()
INFO:httpx:HTTP Request: POST https://gateway.thegraph.com/api/e4b7117cf42e1a57b7d54f3bdd0df9ea/subgraphs/id/DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp "HTTP/2 200 OK"


In [49]:
subgraph_metadata = network_subgraph.Query.subgraphs(
    where=[
        network_subgraph.Subgraph.id == "DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp"
    ]
)

sg.query_df([
    # subgraph_metadata.id,
    subgraph_metadata.metadata.codeRepository,
    subgraph_metadata.currentVersion.subgraphDeployment.ipfsHash
])

INFO:httpx:HTTP Request: POST https://gateway.thegraph.com/api/e4b7117cf42e1a57b7d54f3bdd0df9ea/subgraphs/id/DZz4kDTdmzWLWsV373w2bSmoar3umKKH9y82SUKr5qmp "HTTP/2 200 OK"


,subgraphs_metadata_codeRepository,subgraphs_currentVersion_subgraphDeployment_ipfsHash
0,https://github.com/graphprotocol/graph-network...,QmUzRg2HHMpbgf6Q4VHKNDbtBEJnyp5JWCh2gUX9AV6jXv
